In [1]:
# spark.stop()

In [2]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.transformers import load_and_save_bronze

raw_zone = os.path.expanduser('~/Downloads/dataset.parquet')

df = load_and_save_bronze(raw_zone, './data/bronze/bronze.parquet')
df.show(5)

Saved: ./data/bronze/bronze.parquet
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee| ingestion_timestamp|         source_file|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-----------

In [ ]:
def silver_df(df, num_cols, cat_cols, target):
    selected_cols = num_cols + cat_cols + [target]
    return df.select(*selected_cols)

In [ ]:
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import hour, dayofweek, month

def clean_and_save_silver_df(df):
  df = df.drop("ingestion_timestamp", "source_file")
  df = df.na.drop(how='any')

  df = df.withColumn("trip_duration", 
    (unix_timestamp(col("tpep_dropoff_datetime")) - 
     unix_timestamp(col("tpep_pickup_datetime"))) / 60
  )

  df = df.withColumn("trip_duration", round(col("trip_duration"), 2))
  df = df.filter(col("passenger_count") > 0)

  df = df.filter(col("trip_distance") > 0)
  df = df.filter(col("trip_distance") < 200)
  
  df = df.filter(col("trip_duration") > 0)
  df = df.filter(col("trip_duration") <= 120)


  df = df.filter(
    (col("fare_amount") > 0) &
    (col("fare_amount") <= 500)
  )

  df = df.withColumn("pickuphour", hour("tpep_pickup_datetime")) \
       .withColumn("dayof_week", dayofweek("tpep_pickup_datetime")) \
       .withColumn("month", month("tpep_pickup_datetime"))

  df = df.filter(col("RateCodeID") != 99)

  return df

In [ ]:
df_cleaned = clean_and_save_silver_df(df)

In [ ]:
df_cleaned.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-------------+----------+----------+-----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|trip_duration|pickuphour|dayof_week|month|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-------------+----------+----------+-----+
|   

In [ ]:
df_cleaned.count()

2775092

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LogisticsData") \
    .getOrCreate()

26/01/15 10:14:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
import os
import shutil

def save_as_single_parquet(df, output_filename):
    """
    Saves a Spark DataFrame as a single parquet file with a specific name.
    """
    temp_dir = "temp_spark_out"
    
    df.coalesce(1).write.mode("overwrite").parquet(temp_dir)
    
    part_file = [f for f in os.listdir(temp_dir) if f.startswith("part-") and f.endswith(".parquet")][0]
    
    if os.path.exists(output_filename):
        os.remove(output_filename)
    
    shutil.move(os.path.join(temp_dir, part_file), output_filename)
    
    shutil.rmtree(temp_dir)
    
    print(f"Saved: {output_filename}")


In [ ]:
save_as_single_parquet(df_cleaned, './data/silver/silver.parquet')

Saved: ./data/silver/silver.parquet


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee| ingestion_timestamp|         source_file|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------------+--------------------+
|   

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("App")
    .config(
        "spark.jars",
        "/home/hassan/Desktop/project/simplon/smart-logi-track-API/postgresql-42.7.3.jar"
    )
    .getOrCreate()
)

jdbc_url = "jdbc:postgresql://localhost:5432/smart_logi_track_db"

connection_properties = {
    "user": "postgres",
    "password": "123456",
    "driver": "org.postgresql.Driver"
}

In [ ]:
df_load = spark.read.parquet("./data/silver/")
df_load.write.jdbc(url=jdbc_url, table="silver_data", mode="overwrite", properties=connection_properties)